In [ ]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt

%matplotlib inline

def sliding_windows(image, win_size, win_step):
    '''
    Выделяет скользящие окна из матрицы

    :param image: входное изображение
    :param win_size: размер окна
    :param win_step: шаг, с которым будет перемещаться окно по оси x
    :return: список из подматриц исходной матрицы, которые получились путем прохода окна
    '''
    windows = list()
    h, w, _ = image.shape

    for y in range(0, h - win_size + 1, win_size):
        for x in range(0, w - win_size + 1, win_step):
            win = image[y: y + win_size, x: x + win_size]
            windows.append((x, y, win))  # Теперь возвращаем также координаты (x, y)

    return windows

def is_correct_cell(cell_image):
    '''
    Определяет стоит ли заносить данную ячейку в список тех, что нужно оставить

    :param сell_image: элемент списка скользящих окон
    :return: True если threshold часть ячейки заполнена раковыми образованиями, иначе - false
    '''
    threshold = 0.2 #если такая часть ячейки заполнена раковой опухолью, то запоминаем ее

    total_pixels = cell_image.size

    non_white_sum = np.sum(cell_image < 255) #Считаем все не белые пиксели, если их больше чем threshold*total_pixels, то True

    return non_white_sum > (total_pixels * threshold)
    

def get_cancer_cells(image, cell_size, cell_number):
    '''
    Определяет ячейки в которых содержится больше всего информации о раковых образованиях

    :param image: фото раковых клеток
    :param cell_size: размер ячейки
    :param cell_number: количество ячеек
    :return: список из cell_number ячеек, которые содержат больше всего информации о раковых образованиях
    '''
    suitable_cells = list()

    n = 10

    cell_step = cell_size // n # вместо 10 можно вписать другое число
    cells_list = sliding_windows(image, cell_size, cell_step)

    i = 0
    while i != len(cells_list):
        x, y, cell = cells_list[i]  # Теперь извлекаем координаты (x, y) и саму ячейку
        if(is_correct_cell(cell)):
            suitable_cells.append((x, y, cell))  # Сохраняем координаты и ячейку
            i += n
        else:
            i += 1

    return suitable_cells

# Загрузка изображения
image = cv2.imread('cells/test_train0_1.jpeg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Получение подходящих ячеек
res = get_cancer_cells(image, 150, 10)

# Рисование прямоугольников на исходном изображении
for x, y, cell in res:
    cv2.rectangle(image, (x, y), (x + 150, y + 150), (255, 0, 0), 2)  # Рисуем прямоугольник

# Отображение результата
plt.imshow(image, cmap = 'gray')
plt.axis('off')
plt.show()